In [779]:
import glob
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import Counter
# folderpath = "./f_loc_fsresults_rlog_13/"
# folderpath = "./f_loc_fsresultsNoGBC_rlog_13/"
# folderpath = "./f_steps_fsresults_rlog_6/"
# folderpath = "./f_steps_fsresultsNoGBC_rlog_7/"
# folderpath = "./f_slp_fsresults_rlog_4/"
# folderpath = "./f_slp_fsresultsNoGBC_rlog_10/"
# SENSOR = "f_all7_sloop"
SENSOR = "f_slp_BC"
prebdi_status = "NO"
# modelnumlist = list(set(range(1,26)) - set([8,9,11,13,15,16,17,18,19,21,22,23,24]))
# modelnumlist = list(set(range(1,23)) - set([11]))
modelnumlist = range(1,42)
outcome_status = "change_bdi_2" # change_bdi_2 or post_bdi_2
# outcome_status = "post_bdi_2_levelsC" # change_bdi_2 or post_bdi_2
# outcome_status = "change_bdi_2_levelsC" # change_bdi_2 or post_bdi_2


In [780]:
#utils
def printAllRows(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
        print(df)
def printAllCols(df):
    with pd.option_context('display.max_rows', 3, 'display.max_columns', None):
        print(df)

In [781]:
def runForFolder(folderpath, modeltype, modeli, printtruepreds = False, verbose = True):
    filenames = glob.glob(folderpath+"*_test_model1.csv")
#     print ("Found {0} files".format(len(filenames)))
#     print ("For {0} model # {1}".format(modeltype, modeli))
    testpred_list = []
    testpredproba_list = []
    testtrue_list = []
    did_list = []
    fcnt = 0
    for filename in filenames:
        did = filename.split("_")[-3].split("/")[-1]
        result = pd.read_csv(filename)
        result = result.iloc[0]
        ## We actually dont need the below. Can just get counts from testtrue_list
#         fcnt = fcnt + 1
#         if fcnt == 1:
#             if "cnts_per_lbl" in result:
#                 cnts_per_label = result["cnts_per_lbl"]
#                 print (cnts_per_label)
#             else:
#                 print ("No counts per label found")
        test_proba = result["Y_pred_proba"]
        test_pred = result["Y_pred"]
        test_true = result["Y_true"]
        did_list.append(did)
        testpredproba_list.append(test_proba)
        testpred_list.append(test_pred)
        testtrue_list.append(test_true)
#     print ("Calculating over {0} subjects".format(len(did_list)))
    testpred_list = np.array(testpred_list)
    testtrue_list = np.array(testtrue_list)
    cnts_per_label = dict(Counter(testtrue_list))
    cnts_per_label_str = ','.join("%s:%r" % (key, val) for (key, val) in cnts_per_label.iteritems())
    if printtruepreds:
        df = pd.DataFrame({"Pred": testpred_list, "True": testtrue_list})
        printAllCols(df.T)
    numpple = len(did_list)
    if outcome_status == "post_bdi_2" or outcome_status == "change_bdi_2": # binary
        accuracy = accuracy_score(testtrue_list, testpred_list)
        precision = precision_score(testtrue_list, testpred_list, average = "binary")
        recall = recall_score(testtrue_list, testpred_list, average = "binary")
        f1 = f1_score(testtrue_list, testpred_list, average = "binary")
        if verbose:
            print ("{5} paramDict#{6} \nN = {4}. Cnts_Per_Label = {7}. Accuracy = {0}. Precision = {1}. Recall = {2}. F1 = {3}.".format(accuracy, precision, recall, f1, numpple, modeltype, modeli, cnts_per_label_str))
    elif outcome_status == "post_bdi_2_levelsC" or outcome_status == "change_bdi_2_levelsC": #multiclass
        accuracy = accuracy_score(testtrue_list, testpred_list)
        precision = precision_score(testtrue_list, testpred_list, average = "macro")
        recall = recall_score(testtrue_list, testpred_list, average = "macro")
        f1 = f1_score(testtrue_list, testpred_list, average = "macro")
        wprecision = precision_score(testtrue_list, testpred_list, average = "weighted")
        wrecall = recall_score(testtrue_list, testpred_list, average = "weighted")
        wf1 = f1_score(testtrue_list, testpred_list, average = "weighted")
        if verbose:
            print ("{5} paramDict#{6} \nN = {4}. Cnts_Per_Label = {7}. Accuracy = {0}. Macro Precision = {1}. Macro Recall = {2}. Macro F1 = {3}. Weighted Precision = {8}. Weighted Recall = {9}. Weighted F1 = {10}.".format(accuracy, precision, recall, f1, numpple, modeltype, modeli, cnts_per_label_str, wprecision, wrecall, wf1))
        labelorder = cnts_per_label.keys()
        labelorder_str = ", ".join(str(v) for v in labelorder)
        precisionlbls = precision_score(testtrue_list, testpred_list, labels = labelorder, average = None)
        recalllbls = recall_score(testtrue_list, testpred_list, labels = labelorder, average = None)
        f1lbls = f1_score(testtrue_list, testpred_list, labels = labelorder, average = None)
        precisionlbls_str = ", ".join(str(v) for v in precisionlbls)
        recalllbls_str = ", ".join(str(v) for v in recalllbls)
        f1lbls_str = ", ".join(str(v) for v in f1lbls)
        if verbose: 
            print ("Label order: {0}. Precision: {1}. Recall: {2}. F1: {3}.".format(labelorder_str, precisionlbls_str, recalllbls_str, f1lbls_str))
    return (numpple, accuracy, precision, recall, f1)

def runForOneModel(modeltype, modeli, printtruepreds = False):
    if outcome_status == "post_bdi_2":
        if modeltype == "GBC":
            folderpath = "./models/{1}_fsresultsprebdi{2}_rlog_{0}/".format(modeli, SENSOR, prebdi_status)
        elif modeltype == "LogR" or modeltype == "LOGR":
            folderpath = "./models/{1}_fsresultsNoGBCprebdi{2}_rlog_{0}/".format(modeli, SENSOR, prebdi_status)
        else:
            raise Exception("Model type not recognized")
    elif outcome_status == "change_bdi_2":
        if modeltype == "GBC":
            folderpath = "./models/{1}_outChange_fsresultsprebdi{2}_rlog_{0}/".format(modeli, SENSOR, prebdi_status)
        elif modeltype == "LogR" or modeltype == "LOGR":
            folderpath = "./models/{1}_outChange_fsresultsNoGBCprebdi{2}_rlog_{0}/".format(modeli, SENSOR, prebdi_status)
        else:
            raise Exception("Model type not recognized")
    elif outcome_status == "post_bdi_2_levelsC":
        if modeltype == "GBC":
            folderpath = "./models/{1}_outSeverityC_fsresultsprebdi{2}_rlog_{0}/".format(modeli, SENSOR, prebdi_status)
        elif modeltype == "LogR":
            folderpath = "./models/{1}_outSeverityC_fsresultsNoGBCprebdi{2}_rlog_{0}/".format(modeli, SENSOR, prebdi_status)
        else:
            raise Exception("Model type not recognized")
    elif outcome_status == "change_bdi_2_levelsC":
        if modeltype == "GBC":
            folderpath = "./models/{1}_outChangeLvlsC_fsresultsprebdi{2}_rlog_{0}/".format(modeli, SENSOR, prebdi_status)
        elif modeltype == "LogR":
            folderpath = "./models/{1}_outChangeLvlsC_fsresultsNoGBCprebdi{2}_rlog_{0}/".format(modeli, SENSOR, prebdi_status)
        else:
            raise Exception("Model type not recognized")
    else:
        raise Exception ("Unrecognized outcome_status {0}".format(outcome_status))
    numpple, accuracy, precision, recall, f1 = runForFolder(folderpath, modeltype, modeli, printtruepreds)
    return (numpple, accuracy, precision, recall, f1)


In [782]:
best_acc = 0
best_f1 = 0
best_acc_model = 0
best_f1_model = 0
best_acc_model_type = ""
best_f1_model_type = ""
print (modelnumlist)

# GBC model
for i in modelnumlist:
    numpple, accuracy, precision, recall, f1 = runForOneModel("GBC", i)
    if accuracy>=best_acc:
        best_acc = accuracy
        best_acc_model = i
        best_acc_model_type = "GBC"
    if f1>=best_f1:
        best_f1 = f1
        best_f1_model = i
        best_f1_model_type = "GBC"

# LogR model
for i in modelnumlist:
    numpple, accuracy, precision, recall, f1 = runForOneModel("LogR", i)
    if accuracy>best_acc:
        best_acc = accuracy
        best_acc_model = i
        best_acc_model_type = "LogR"
    if f1>best_f1:
        best_f1 = f1
        best_f1_model = i
        best_f1_model_type = "LogR"

print ("\nLooking at Accuracy, BEST is {0} model #{1}".format(best_acc_model_type, best_acc_model))
numpple, accuracy, precision, recall, f1 = runForOneModel(best_acc_model_type, best_acc_model, False)

print ("\nLooking at F1, BEST is {0} model #{1}".format(best_f1_model_type, best_f1_model))
numpple, accuracy, precision, recall, f1 = runForOneModel(best_f1_model_type, best_f1_model, False)





[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]
GBC paramDict#1 
N = 107. Cnts_Per_Label = 0:67,1:40. Accuracy = 0.560747663551. Precision = 0.393939393939. Recall = 0.325. F1 = 0.356164383562.
GBC paramDict#2 
N = 107. Cnts_Per_Label = 0:67,1:40. Accuracy = 0.570093457944. Precision = 0.421052631579. Recall = 0.4. F1 = 0.410256410256.
GBC paramDict#3 
N = 107. Cnts_Per_Label = 0:67,1:40. Accuracy = 0.514018691589. Precision = 0.35. Recall = 0.35. F1 = 0.35.
GBC paramDict#4 
N = 107. Cnts_Per_Label = 0:67,1:40. Accuracy = 0.532710280374. Precision = 0.361111111111. Recall = 0.325. F1 = 0.342105263158.
GBC paramDict#5 
N = 107. Cnts_Per_Label = 0:67,1:40. Accuracy = 0.560747663551. Precision = 0.4. Recall = 0.35. F1 = 0.373333333333.
GBC paramDict#6 
N = 107. Cnts_Per_Label = 0:67,1:40. Accuracy = 0.570093457944. Precision = 0.416666666667. Recall = 0.375. F1 = 0.394736842105.
GBC

LogR paramDict#17 
N = 107. Cnts_Per_Label = 0:67,1:40. Accuracy = 0.504672897196. Precision = 0.372549019608. Recall = 0.475. F1 = 0.417582417582.
LogR paramDict#18 
N = 107. Cnts_Per_Label = 0:67,1:40. Accuracy = 0.551401869159. Precision = 0.409090909091. Recall = 0.45. F1 = 0.428571428571.
LogR paramDict#19 
N = 107. Cnts_Per_Label = 0:67,1:40. Accuracy = 0.542056074766. Precision = 0.411764705882. Recall = 0.525. F1 = 0.461538461538.
LogR paramDict#20 
N = 107. Cnts_Per_Label = 0:67,1:40. Accuracy = 0.504672897196. Precision = 0.372549019608. Recall = 0.475. F1 = 0.417582417582.
LogR paramDict#21 
N = 107. Cnts_Per_Label = 0:67,1:40. Accuracy = 0.560747663551. Precision = 0.436363636364. Recall = 0.6. F1 = 0.505263157895.
LogR paramDict#22 
N = 107. Cnts_Per_Label = 0:67,1:40. Accuracy = 0.504672897196. Precision = 0.377358490566. Recall = 0.5. F1 = 0.430107526882.
LogR paramDict#23 
N = 107. Cnts_Per_Label = 0:67,1:40. Accuracy = 0.551401869159. Precision = 0.423076923077. Recall